### 162 latest 6month feature
season2のfeature setでは、state, cityなどを使わず、category_2を使う

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [72]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
# df_trans = utils.read_df_pkl('../input/trans_last1*')
# df_trans = utils.read_df_pkl('../input/trans_last2*')
# df_trans = utils.read_df_pkl('../input/trans_last3*')
df_trans = utils.read_df_pkl('../input/trans_last4*')
df_trans = reduce_mem_usage(df_trans)

auth1 = df_trans[df_trans.authorized_flag==1]
auth0 = df_trans[df_trans.authorized_flag==0]

auth1_cat1 = auth1[auth1.category_1==1]
auth1_cat0 = auth1[auth1.category_1==0]
auth0_cat1 = auth0[auth0.category_1==1]
auth0_cat0 = auth0[auth0.category_1==0]
del auth1, auth0
gc.collect()

100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


Mem. usage decreased to 489.29 Mb (34.8% reduction)


7198

### Null Count

In [73]:
last_type = 'last1'
last_type = 'last2'
last_type = 'last3'
last_type = 'last4'
null_cols = ['category_2', 'installments', 'merchant_id']

# category_2
col = 'category_2'
fname = 'auth1_cat0'
auth1_cat0[f'{col}_null_cnt'] = (auth1_cat0[col]=='NA')
fname = 'auth0_cat0'
auth0_cat0[f'{col}_null_cnt'] = (auth0_cat0[col]=='NA')

# installments
col = 'installments'
fname = 'auth1_cat0'
auth1_cat0[f'{col}_isnull'] = (auth1_cat0[col].isnull())
fname = 'auth1_cat1'
auth1_cat1[f'{col}_isnull'] = (auth1_cat1[col].isnull())
fname = 'auth0_cat0'
auth0_cat0[f'{col}_isnull'] = (auth0_cat0[col].isnull())
fname = 'auth0_cat1'
auth0_cat1[f'{col}_isnull'] = (auth0_cat1[col].isnull())

# merchant_id
col = 'merchant_id'
fname = 'auth1_cat0'
auth1_cat0[f'{col}_isnull'] = (auth1_cat0[col].isnull())
fname = 'auth1_cat1'
auth1_cat1[f'{col}_isnull'] = (auth1_cat1[col].isnull())
fname = 'auth0_cat0'
auth0_cat0[f'{col}_isnull'] = (auth0_cat0[col].isnull())
fname = 'auth0_cat1'
auth0_cat1[f'{col}_isnull'] = (auth0_cat1[col].isnull())

In [74]:
last_type = 'last1'
last_type = 'last2'
last_type = 'last3'
last_type = 'last4'
df_list = [auth1_cat1, auth1_cat0, auth0_cat1, auth0_cat0]
fname_list = [f'{last_type}_auth1_cat1', f'{last_type}_auth1_cat0', f'{last_type}_auth0_cat1', f'{last_type}_auth0_cat0']
train_list = []
test_list = []

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


def feat_agg(df, fname):

    null_cols = [col for col in df.columns if col.count('isnull')]
    
    aggs = {}
    
    # category_1==1のときcategory_2 = Null
    if fname.count('cat0'):
        cat_list = ['category_2']
        df = get_dummies(df=df, cat_list=cat_list)
        category_2_list = [col for col in df.columns if col.count('category_2')]
        for cat in category_2_list:
            aggs[cat] = ['mean', 'sum']
            
    # isnull feature
    for col in null_cols:
        aggs[col] = ['mean', 'sum']
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    df['unix_date'] =  df['purchase_date'].map(lambda x: int(x.timestamp()) )
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum','max', 'min','mean', 'std']
    aggs['unix_date'] = ['max','min']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    aggs['card_id'] = ['size']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
    
    df_agg[f'{fname}_term'] = (df_agg[f'{fname}_unix_date_max'] - df_agg[f'{fname}_unix_date_min']).astype('int64')
    df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
    df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
    df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / df_agg[f'{fname}_installments_sum']
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / df_agg[f'{fname}_installments_mean']
    
    feat_train = df_train.merge(df_agg, on=key, how='left')
    feat_test = df_test.merge(df_agg, on=key, how='left')
    del df_agg
    gc.collect()
    
    print(fname)
    display(feat_train.head())
    
    train_list.append(feat_train)
    test_list.append(feat_test)
    
for df, fname in zip(df_list, fname_list):
    feat_agg(df, fname)

last4_auth1_cat1


,card_id,first_active_month,feature_1,feature_2,feature_3,target,last4_auth1_cat1_installments_isnull_mean,last4_auth1_cat1_installments_isnull_sum,last4_auth1_cat1_merchant_id_isnull_mean,last4_auth1_cat1_merchant_id_isnull_sum,last4_auth1_cat1_month_lag_mean,last4_auth1_cat1_month_lag_std,last4_auth1_cat1_yyyy_week_nunique,last4_auth1_cat1_purchase_amount_sum,last4_auth1_cat1_purchase_amount_max,last4_auth1_cat1_purchase_amount_min,last4_auth1_cat1_purchase_amount_mean,last4_auth1_cat1_purchase_amount_over_sum,last4_auth1_cat1_purchase_amount_over_max,last4_auth1_cat1_purchase_amount_over_min,last4_auth1_cat1_purchase_amount_over_mean,last4_auth1_cat1_installments_sum,last4_auth1_cat1_installments_max,last4_auth1_cat1_installments_min,last4_auth1_cat1_installments_mean,last4_auth1_cat1_installments_std,last4_auth1_cat1_unix_date_max,last4_auth1_cat1_unix_date_min,last4_auth1_cat1_purchase_date_max,last4_auth1_cat1_purchase_date_min,last4_auth1_cat1_month_diff_mean,last4_auth1_cat1_month_diff_std,last4_auth1_cat1_card_id_size,last4_auth1_cat1_term,last4_auth1_cat1_freq_per_90days,last4_auth1_cat1_amount_per_90days,last4_auth1_cat1_installments_per_90days,last4_auth1_cat1_amount_per_installments_sum,last4_auth1_cat1_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


last4_auth1_cat0


,card_id,first_active_month,feature_1,feature_2,feature_3,target,last4_auth1_cat0_category_2_null_cnt_mean,last4_auth1_cat0_category_2_null_cnt_sum,last4_auth1_cat0_dummie_category_2_1.0_mean,last4_auth1_cat0_dummie_category_2_1.0_sum,last4_auth1_cat0_dummie_category_2_2.0_mean,last4_auth1_cat0_dummie_category_2_2.0_sum,last4_auth1_cat0_dummie_category_2_3.0_mean,last4_auth1_cat0_dummie_category_2_3.0_sum,last4_auth1_cat0_dummie_category_2_4.0_mean,last4_auth1_cat0_dummie_category_2_4.0_sum,last4_auth1_cat0_dummie_category_2_5.0_mean,last4_auth1_cat0_dummie_category_2_5.0_sum,last4_auth1_cat0_installments_isnull_mean,last4_auth1_cat0_installments_isnull_sum,last4_auth1_cat0_merchant_id_isnull_mean,last4_auth1_cat0_merchant_id_isnull_sum,last4_auth1_cat0_month_lag_mean,last4_auth1_cat0_month_lag_std,last4_auth1_cat0_yyyy_week_nunique,last4_auth1_cat0_purchase_amount_sum,last4_auth1_cat0_purchase_amount_max,last4_auth1_cat0_purchase_amount_min,last4_auth1_cat0_purchase_amount_mean,last4_auth1_cat0_purchase_amount_over_sum,last4_auth1_cat0_purchase_amount_over_max,last4_auth1_cat0_purchase_amount_over_min,last4_auth1_cat0_purchase_amount_over_mean,last4_auth1_cat0_installments_sum,last4_auth1_cat0_installments_max,last4_auth1_cat0_installments_min,last4_auth1_cat0_installments_mean,last4_auth1_cat0_installments_std,last4_auth1_cat0_unix_date_max,last4_auth1_cat0_unix_date_min,last4_auth1_cat0_purchase_date_max,last4_auth1_cat0_purchase_date_min,last4_auth1_cat0_month_diff_mean,last4_auth1_cat0_month_diff_std,last4_auth1_cat0_card_id_size,last4_auth1_cat0_term,last4_auth1_cat0_freq_per_90days,last4_auth1_cat0_amount_per_90days,last4_auth1_cat0_installments_per_90days,last4_auth1_cat0_amount_per_installments_sum,last4_auth1_cat0_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,False,False,1.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,-7.058824,0.237635,6.0,3.626953,0.439697,0.015038,0.071106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,1.501509e+09,1.498573e+09,2017-07-31 13:43:15,2017-06-27 14:18:08,290.784314,9.146177,51.0,2935507.0,0.566667,0.040314,0.00000,inf,inf
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,False,False,1.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,-7.757576,0.860422,13.0,7.070312,0.824707,0.009026,0.107117,0.0,0.0,0.0,0.0,101.0,10.0,1.0,1.530273,1.53125,1.498741e+09,1.491082e+09,2017-06-29 13:01:50,2017-04-01 21:30:09,342.651515,28.526097,66.0,7659101.0,0.733333,0.078552,1.12207,0.070007,0.070007
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,False,False,0.0,0.0,-8.333333,1.032796,3.0,0.338379,0.092712,0.031464,0.056396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,1.500654e+09,1.494690e+09,2017-07-21 16:28:16,2017-05-13 15:31:59,323.333333,31.557355,6.0,5964977.0,0.066667,0.003759,0.00000,inf,inf
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


last4_auth0_cat1


,card_id,first_active_month,feature_1,feature_2,feature_3,target,last4_auth0_cat1_installments_isnull_mean,last4_auth0_cat1_installments_isnull_sum,last4_auth0_cat1_merchant_id_isnull_mean,last4_auth0_cat1_merchant_id_isnull_sum,last4_auth0_cat1_month_lag_mean,last4_auth0_cat1_month_lag_std,last4_auth0_cat1_yyyy_week_nunique,last4_auth0_cat1_purchase_amount_sum,last4_auth0_cat1_purchase_amount_max,last4_auth0_cat1_purchase_amount_min,last4_auth0_cat1_purchase_amount_mean,last4_auth0_cat1_purchase_amount_over_sum,last4_auth0_cat1_purchase_amount_over_max,last4_auth0_cat1_purchase_amount_over_min,last4_auth0_cat1_purchase_amount_over_mean,last4_auth0_cat1_installments_sum,last4_auth0_cat1_installments_max,last4_auth0_cat1_installments_min,last4_auth0_cat1_installments_mean,last4_auth0_cat1_installments_std,last4_auth0_cat1_unix_date_max,last4_auth0_cat1_unix_date_min,last4_auth0_cat1_purchase_date_max,last4_auth0_cat1_purchase_date_min,last4_auth0_cat1_month_diff_mean,last4_auth0_cat1_month_diff_std,last4_auth0_cat1_card_id_size,last4_auth0_cat1_term,last4_auth0_cat1_freq_per_90days,last4_auth0_cat1_amount_per_90days,last4_auth0_cat1_installments_per_90days,last4_auth0_cat1_amount_per_installments_sum,last4_auth0_cat1_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,False,False,0.0,0.0,-8.0,NaN,1.0,1.009766,1.009766,1.009766,1.009766,0.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,NaN,1.494244e+09,1.494244e+09,2017-05-08 11:43:11,2017-05-08 11:43:11,357.0,NaN,1.0,0.0,0.011111,0.011223,0.111084,0.100952,0.100952
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


last4_auth0_cat0


,card_id,first_active_month,feature_1,feature_2,feature_3,target,last4_auth0_cat0_category_2_null_cnt_mean,last4_auth0_cat0_category_2_null_cnt_sum,last4_auth0_cat0_dummie_category_2_1.0_mean,last4_auth0_cat0_dummie_category_2_1.0_sum,last4_auth0_cat0_dummie_category_2_2.0_mean,last4_auth0_cat0_dummie_category_2_2.0_sum,last4_auth0_cat0_dummie_category_2_3.0_mean,last4_auth0_cat0_dummie_category_2_3.0_sum,last4_auth0_cat0_dummie_category_2_4.0_mean,last4_auth0_cat0_dummie_category_2_4.0_sum,last4_auth0_cat0_dummie_category_2_5.0_mean,last4_auth0_cat0_dummie_category_2_5.0_sum,last4_auth0_cat0_installments_isnull_mean,last4_auth0_cat0_installments_isnull_sum,last4_auth0_cat0_merchant_id_isnull_mean,last4_auth0_cat0_merchant_id_isnull_sum,last4_auth0_cat0_month_lag_mean,last4_auth0_cat0_month_lag_std,last4_auth0_cat0_yyyy_week_nunique,last4_auth0_cat0_purchase_amount_sum,last4_auth0_cat0_purchase_amount_max,last4_auth0_cat0_purchase_amount_min,last4_auth0_cat0_purchase_amount_mean,last4_auth0_cat0_purchase_amount_over_sum,last4_auth0_cat0_purchase_amount_over_max,last4_auth0_cat0_purchase_amount_over_min,last4_auth0_cat0_purchase_amount_over_mean,last4_auth0_cat0_installments_sum,last4_auth0_cat0_installments_max,last4_auth0_cat0_installments_min,last4_auth0_cat0_installments_mean,last4_auth0_cat0_installments_std,last4_auth0_cat0_unix_date_max,last4_auth0_cat0_unix_date_min,last4_auth0_cat0_purchase_date_max,last4_auth0_cat0_purchase_date_min,last4_auth0_cat0_month_diff_mean,last4_auth0_cat0_month_diff_std,last4_auth0_cat0_card_id_size,last4_auth0_cat0_term,last4_auth0_cat0_freq_per_90days,last4_auth0_cat0_amount_per_90days,last4_auth0_cat0_installments_per_90days,last4_auth0_cat0_amount_per_installments_sum,last4_auth0_cat0_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,False,False,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,-7.0,NaN,1.0,0.314941,0.314941,0.314941,0.314941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.500131e+09,1.500131e+09,2017-07-15 14:58:10,2017-07-15 14:58:10,289.0,NaN,1.0,0.0,0.011111,0.0035,0.0,inf,inf
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# impute

def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
        
for train, test in zip(train_list, test_list):
    try:
        train.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month'], axis=1, inplace=True)
    except KeyError:
        pass
    
    try:
        train.set_index(key, inplace=True)
        test.set_index(key, inplace=True)
    except KeyError:
        pass
    
    df_train = df_train.join(train)
    df_test = df_test.join(test)

df_train = impute_agg(df_train)
df_test = impute_agg(df_test)

In [76]:
last_type = 'last1'
last_type = 'last2'
last_type = 'last3'
last_type = 'last4'

df_train['first_active_month'] =  pd.to_datetime(df_train['first_active_month'])
df_test['first_active_month'] =  pd.to_datetime(df_test['first_active_month'])

# 最終のcat0, auth1までの期間
df_train[f'{last_type}_auth1_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_auth1_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# 最終のcat1, auth1までの期間
df_train[f'{last_type}_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_elapsed_time_from_first'] = (df_train[f'{last_type}_auth1_cat1_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_elapsed_time_from_first']  = (df_test[f'{last_type}_auth1_cat1_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# {last_type}の初回購買までの期間
df_train[f'{last_type}_cat0_first_buy'] = (df_train[f'{last_type}_auth1_cat0_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_first_buy']  = (df_test[f'{last_type}_auth1_cat0_purchase_date_min'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_first_buy'] = (df_train[f'{last_type}_auth1_cat1_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_first_buy']  = (df_test[f'{last_type}_auth1_cat1_purchase_date_min'] - df_test[f'first_active_month']).dt.days

# 20180501までの期間を各データセットパターンで
df_train[f'{last_type}_auth1_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth1_cat0_purchase_date_max']).dt.days
df_train[f'{last_type}_auth1_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth1_cat1_purchase_date_max']).dt.days
df_train[f'{last_type}_auth0_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth0_cat0_purchase_date_max']).dt.days
df_train[f'{last_type}_auth0_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_auth0_cat1_purchase_date_max']).dt.days
df_test[f'{last_type}_auth1_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth1_cat0_purchase_date_max']).dt.days
df_test[f'{last_type}_auth1_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth1_cat1_purchase_date_max']).dt.days
df_test[f'{last_type}_auth0_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth0_cat0_purchase_date_max']).dt.days
df_test[f'{last_type}_auth0_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_auth0_cat1_purchase_date_max']).dt.days

# cat0, auth1, auth0の間の期間
df_train[f'{last_type}_auth1_0_term'] = df_train[f'{last_type}_auth1_cat0_term_from_now'] - df_train[f'{last_type}_auth0_cat0_term_from_now']
df_test[f'{last_type}_auth1_0_term']  = df_test[f'{last_type}_auth1_cat0_term_from_now'] - df_test[f'{last_type}_auth0_cat0_term_from_now']

if last_type=='last1':
    df_train['unix_first_active_month'] =  df_train['first_active_month'].map(lambda x: x.timestamp())
    df_test['first_active_month'].fillna('2017-10', inplace=True)
    df_test['first_active_month'] =  pd.to_datetime(df_test['first_active_month'])
    df_test['unix_first_active_month'] =  df_test['first_active_month'].map(lambda x: x.timestamp())

In [77]:
# Null Impute
for col in df_train.columns:
    if col in ignore_list:continue
    if str(type(df_test[col].dtypes)).count('time'):
        df_train[col].fillna(df_train[col].min(), inplace=True)
        df_test[col].fillna(df_train[col].min(), inplace=True)

### Save Feature

In [78]:
fname = '162_fe2'
ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
    feat_train = df_train[col].astype('float32').values
    try:
        feat_test = df_test[col].astype('float32').values
    except TypeError:
        print(col)
        sys.exit()
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)

In [71]:
import glob
path_list = glob.glob('../features/1_first_valid/*.gz')
for path in path_list:
    length = utils.read_pkl_gzip(path).shape[0]
    if path.count('test'):
        print(length)
#         if length>220000:
#             print(path)
#     if path.count('test'):
#         if length>140000:
#             print(path)

SystemExit: 

In [80]:
auth1_cat1.head()

,card_id,month_lag,all_term,authorized_flag,category_1,category_2,category_3,city_id,first_active_month,installments,merchant_category_id,merchant_id,purchase_amount,purchase_amount_over,purchase_date,state_id,subsector_id,weekofyear,yyyy_week,yyyymm,yyyymmdd,installments_isnull,merchant_id_isnull,month_diff,unix_date
46,C_ID_00007093c1,-8,454.0,1.0,1.0,NaN,B,-1.0,2017-02-01,1.0,755.0,M_ID_57df19bf28,0.025558,0.0,2017-06-14 14:22:05,-1.0,8.0,24.0,2017-24,2017-06,2017-06-14,False,False,320,1497450125
56,C_ID_00007093c1,-7,454.0,1.0,1.0,NaN,B,-1.0,2017-02-01,1.0,755.0,M_ID_57df19bf28,0.037567,0.0,2017-07-03 15:43:57,-1.0,8.0,27.0,2017-27,2017-07,2017-07-03,False,False,301,1499096637
59,C_ID_00007093c1,-7,454.0,1.0,1.0,NaN,B,-1.0,2017-02-01,1.0,755.0,M_ID_57df19bf28,0.025558,0.0,2017-07-14 11:51:05,-1.0,8.0,28.0,2017-28,2017-07,2017-07-14,False,False,290,1500033065
68,C_ID_00007093c1,-7,454.0,1.0,1.0,NaN,B,-1.0,2017-02-01,1.0,755.0,M_ID_57df19bf28,0.025558,0.0,2017-07-06 10:58:03,-1.0,8.0,27.0,2017-27,2017-07,2017-07-06,False,False,298,1499338683
69,C_ID_00007093c1,-7,454.0,1.0,1.0,NaN,B,-1.0,2017-02-01,1.0,755.0,M_ID_57df19bf28,0.025558,0.0,2017-07-31 16:01:04,-1.0,8.0,31.0,2017-31,2017-07,2017-07-31,False,False,273,1501516864
